#  Download datasets

In [1]:
!pip install gdown

In [2]:
!gdown 1WZ5BhVuhNDvur-wo_OnfcUGB16HcU7cR

Downloading...
From (original): https://drive.google.com/uc?id=1WZ5BhVuhNDvur-wo_OnfcUGB16HcU7cR
From (redirected): https://drive.google.com/uc?id=1WZ5BhVuhNDvur-wo_OnfcUGB16HcU7cR&confirm=t&uuid=408480aa-33d3-4f95-8200-a25181fc5591
To: /content/LibriSpeech.zip
100% 7.08G/7.08G [01:38<00:00, 71.8MB/s]


In [3]:
!unzip LibriSpeech.zip

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0049.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0050.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0051.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0052.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0053.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0054.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0055.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0056.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0057.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0058.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0059.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0060.flac  
  inflating: LibriSpeech/train-clean/7447/91187/7447-91187-0061.flac  
  inflating: Libri

# Import Library

In [17]:
import os
import glob
import shutil
from pydub import AudioSegment
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [11]:
SOURCE_FOLDER = "LibriSpeech"
OUTPUT_FOLDER = "LibriSpeech_WAV"

# Utility Function to Convert and Extract Files

In [27]:
def convert_audio(file_path):
    """
      Mengonversi file audio format FLAC ke WAV dengan sample rate 16kHz
      dan menyimpannya ke struktur direktori yang dicerminkan (mirrored).

      Args:
          file_path (str): Path lengkap (absolute/relative) menuju file .flac sumber.

      Global Constants yang digunakan:
          SOURCE_FOLDER (str): Path root folder asal (misal: 'Librispeech').
          OUTPUT_FOLDER (str): Path root folder tujuan (misal: 'Librispeech_WAV').

      Returns:
          None. Fungsi ini tidak mengembalikan nilai, tetapi menulis file ke disk.

      Raises:
          Exception: Menangkap error apa pun saat proses I/O atau decoding audio
                    dan mencetaknya ke konsol tanpa menghentikan program.
    """
    try:
        rel_path = os.path.relpath(file_path, SOURCE_FOLDER)

        rel_path_wav = os.path.splitext(rel_path)[0] + ".wav"
        output_path = os.path.join(OUTPUT_FOLDER, rel_path_wav)

        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        if os.path.exists(output_path):
            return

        audio = AudioSegment.from_file(file_path, format="flac")
        audio = audio.set_frame_rate(16000)
        audio.export(output_path, format="wav")

    except Exception as e:
        print(f"Error converting {file_path}: {e}")

In [26]:
def copy_transcript(file_path):
    """
      Menyalin file transkrip (.txt) dari folder sumber ke folder tujuan
      dengan mempertahankan struktur direktori asli.

      Args:
          file_path (str): Path lengkap menuju file .txt sumber.

      Global Constants yang digunakan:
          SOURCE_FOLDER (str): Path root folder asal.
          OUTPUT_FOLDER (str): Path root folder tujuan.

      Returns:
          None.
    """
    try:
        rel_path = os.path.relpath(file_path, SOURCE_FOLDER)
        output_path = os.path.join(OUTPUT_FOLDER, rel_path)

        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        if not os.path.exists(output_path):
            shutil.copy2(file_path, output_path)

    except Exception as e:
        print(f"Error copying {file_path}: {e}")

# Main Process to Convert Into New Dataset

In [22]:
def main():
    if not os.path.exists(SOURCE_FOLDER):
        print(f"Error: Folder '{SOURCE_FOLDER}' tidak ditemukan!")
        return

    print("--- TAHAP 1: MENGONVERSI AUDIO (.flac -> .wav) ---")
    flac_files = glob.glob(os.path.join(SOURCE_FOLDER, "**", "*.flac"), recursive=True)

    if flac_files:
        print(f"Ditemukan {len(flac_files)} file audio. Memproses...")
        with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
            list(tqdm(executor.map(convert_audio, flac_files), total=len(flac_files), unit="file"))
    else:
        print("Tidak ada file .flac ditemukan.")

    print("\n")

    print("--- TAHAP 2: MENYALIN TRANSCRIPT (.txt) ---")
    txt_files = glob.glob(os.path.join(SOURCE_FOLDER, "**", "*.txt"), recursive=True)

    if txt_files:
        print(f"Ditemukan {len(txt_files)} file transcript. Menyalin...")
        for txt_file in tqdm(txt_files, unit="file"):
            copy_transcript(txt_file)
    else:
        print("Tidak ada file transcript (.txt) ditemukan.")

    print(f"\nSelesai! Semua data tersimpan di folder '{OUTPUT_FOLDER}'")

In [23]:
main()

--- TAHAP 1: MENGONVERSI AUDIO (.flac -> .wav) ---
Ditemukan 33862 file audio. Memproses...


100%|██████████| 33862/33862 [1:41:53<00:00,  5.54file/s]




--- TAHAP 2: MENYALIN TRANSCRIPT (.txt) ---
Ditemukan 769 file transcript. Menyalin...


100%|██████████| 769/769 [00:01<00:00, 703.22file/s]


Selesai! Semua data tersimpan di folder 'LibriSpeech_WAV'


In [25]:
!zip -r LibriSpeech_WAV.zip LibriSpeech_WAV

Output streaming akan dipotong hingga 5000 baris terakhir.
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0008.wav (deflated 21%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0029.wav (deflated 18%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0003.wav (deflated 18%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0019.wav (deflated 17%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0000.wav (deflated 21%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0012.wav (deflated 18%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0020.wav (deflated 18%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0014.wav (deflated 18%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0011.wav (deflated 20%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0022.wav (deflated 16%)
  adding: LibriSpeech_WAV/train-clean/7517/100442/7517-100442-0028.wav (deflated 

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!cp LibriSpeech_WAV.zip /content/drive/MyDrive/